# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

In [38]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv')
def standardize_columns(df):
    df.columns = (
        df.columns.str.strip()
            .str.lower()
            .str.replace(r"\s", "_", regex=True)
    )
    return df
df = standardize_columns(df)

df.dropna(how='all', inplace=True)
df.dropna(axis=1, how='all', inplace=True)

df[df.select_dtypes(include='float').columns] = df.select_dtypes(include='float').round(2)

df.drop('unnamed:_0', axis=1, inplace=True)

df = df.loc[lambda x: (x['total_claim_amount'] > 1000) & (x['response'].str.lower() == 'yes')]

df

,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,income,...,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type
189,OK31456,California,11009.13,Yes,Premium,Bachelor,1/24/11,Employed,F,51643,...,0.0,1,Corporate Auto,Corporate L3,Offer2,Agent,1358.40,Luxury Car,Medsize,NaN
236,YJ16163,Oregon,11009.13,Yes,Premium,Bachelor,1/24/11,Employed,F,51643,...,0.0,1,Special Auto,Special L3,Offer2,Agent,1358.40,Luxury Car,Medsize,A
419,GW43195,Oregon,25807.06,Yes,Extended,College,2/13/11,Employed,F,71210,...,1.0,2,Personal Auto,Personal L2,Offer1,Branch,1027.20,Luxury Car,Small,A
442,IP94270,Arizona,13736.13,Yes,Premium,Master,2/13/11,Disabled,F,16181,...,0.0,8,Personal Auto,Personal L2,Offer1,Web,1261.32,SUV,Medsize,A
587,FJ28407,California,5619.69,Yes,Premium,High School or Below,1/26/11,Unemployed,M,0,...,0.0,1,Personal Auto,Personal L1,Offer2,Web,1027.00,SUV,Medsize,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10351,FN44127,Oregon,3508.57,Yes,Extended,College,1/5/11,Medical Leave,M,20978,...,1.0,1,Personal Auto,Personal L2,Offer2,Branch,1176.28,Four-Door Car,Small,NaN
10373,XZ64172,Oregon,10963.96,Yes,Premium,High School or Below,2/8/11,Employed,M,55687,...,0.0,1,Corporate Auto,Corporate L2,Offer1,Agent,1324.80,Luxury SUV,Medsize,NaN
10487,IX60941,Oregon,3508.57,Yes,Extended,College,1/5/11,Medical Leave,M,20978,...,1.0,1,Personal Auto,Personal L3,Offer2,Branch,1176.28,Four-Door Car,Small,NaN
10565,QO62792,Oregon,7840.17,Yes,Extended,College,1/14/11,Employed,M,58414,...,2.0,1,Personal Auto,Personal L3,Offer2,Agent,1008.00,NaN,NaN,NaN


2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

In [40]:
df2 = (df[df["response"].str.lower() == "yes"].groupby(["policy_type", "gender"])["total_claim_amount"]
        .mean()
        .round(2)
        .reset_index()
        .sort_values(by="total_claim_amount", ascending=True))
df2

,policy_type,gender,total_claim_amount
5,Special Auto,M,1017.50
3,Personal Auto,M,1137.86
0,Corporate Auto,F,1138.40
1,Corporate Auto,M,1171.15
2,Personal Auto,F,1214.85
4,Special Auto,F,1358.40


3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

In [60]:
pivot = df.drop_duplicates(subset="customer").pivot_table(
    index='state',
    values="customer",
    aggfunc="count"
).reset_index()

pivot = pivot.rename(columns={"customer": "customer_count"})
df3 = pivot.loc[pivot["customer_count"] > 500]
df3 = df3.sort_values(by="customer_count", ascending=False)

print(df3)

Empty DataFrame
Columns: [state, customer_count]
Index: []


4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [62]:
df4 = df.groupby(["education", "gender"])["customer_lifetime_value"].agg(['max', 'min', 'median']).reset_index()
df4[['max', 'min', 'median']] = df4[['max', 'min', 'median']].round(2)
df4

,education,gender,max,min,median
0,Bachelor,F,11009.13,8427.17,10179.72
1,College,F,25807.06,25807.06,25807.06
2,College,M,7840.17,3508.57,5674.37
3,High School or Below,M,10963.96,5619.69,8291.82
4,Master,F,19160.99,13736.13,16448.56


## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [ ]:
# your code goes here